In [36]:
import os
import sys
import requests
import json
from langchain_groq import ChatGroq


current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
# Append the current and parent directories to the system path
sys.path.append(current_dir)
sys.path.append(parent_dir)

from src.config import settings, logger

os.environ["GROQ_API_KEY"] = settings.GROQ_API_KEY

### Create LLM using Groq

In [19]:
llm = ChatGroq(
    model=settings.MODEL_NAME,
    temperature=settings.TEMPERATURE,
    max_tokens=settings.MAX_TOKENS,
    reasoning_format="parsed",
    timeout=settings.TIMEOUT,
    max_retries=settings.MAX_RETERIES,
)


In [37]:
# Test
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence. "
        "Then give me word to word translations",
    ),
    ("human", "I love programming and Pandas."),
]
ai_msg = llm.invoke(messages)
print(ai_msg.content)

2025-08-10 22:38:03,772 - httpx - INFO - _client.py - _send_single_request:1025 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


The translation of the sentence "I love programming and Pandas." into French is "J'aime la programmation et Pandas." Here is the word-to-word breakdown:

1. I - J'
2. love - aime
3. programming - la programmation
4. and - et
5. Pandas - Pandas

This breakdown accurately reflects each word's translation, maintaining the proper structure and meaning in French.


### Tools

In [38]:
from langchain_core.tools import tool

@tool
def get_weather_forcast(loc: str):
    """
    Get the weather forcast of the given day
    :param loc: string loc. Send the input as {'loc': 'loc_value'}
    :return: the weather forcast
    """
    try:
        url = f"https://wttr.in/{loc}?format=j1"
        response = requests.get(url, timeout=20)
        weather_data = response.json()
        area = weather_data["nearest_area"][0]
        area_name = area["areaName"][0]["value"]
        country = area["country"][0]["value"]
        region = area.get("region", [{"value": ""}])[0]["value"]

        location_str = f"{area_name}"
        if region and region != area_name:
            location_str += f", {region}"
        location_str += f", {country}"

        forecast_report = f"📅 **{3}-Day Weather Forecast for {location_str}**\n\n"

        for i, day in enumerate(weather_data["weather"][:3]):
            date = day["date"]
            max_temp_c = day["maxtempC"]
            max_temp_f = day["maxtempF"]
            min_temp_c = day["mintempC"]
            min_temp_f = day["mintempF"]

            # Get the most representative weather description (usually around midday)
            hourly = day["hourly"]
            midday_weather = hourly[len(hourly)//2] if hourly else hourly[0]
            weather_desc = midday_weather["weatherDesc"][0]["value"]

            day_label = "Today" if i == 0 else ("Tomorrow" if i == 1 else date)

            forecast_report += f"**{day_label} ({date}):**\n"
            forecast_report += f"🌡️ High: {max_temp_c}°C ({max_temp_f}°F) | Low: {min_temp_c}°C ({min_temp_f}°F)\n"
            forecast_report += f"☁️ {weather_desc}\n\n"

        # print(forecast_report)
        return forecast_report
    except Exception as e:
        return (f"Error happened during get_weather_forcast call: {e}"
                f"Check the error and if possible, try again.")
        

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

### Create react agent

In [41]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(
    model=llm,
    tools=[multiply, get_weather_forcast]
)
response = agent.invoke({"messages": [{"role": "user", "content": "what's 42 x 7?"}]})
response


2025-08-10 22:39:28,546 - httpx - INFO - _client.py - _send_single_request:1025 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-10 22:39:29,226 - httpx - INFO - _client.py - _send_single_request:1025 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-10 22:39:30,428 - httpx - INFO - _client.py - _send_single_request:1025 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


{'messages': [HumanMessage(content="what's 42 x 7?", additional_kwargs={}, response_metadata={}, id='9541f819-2e47-4579-a8ee-7aeeb9bbc32d'),
  AIMessage(content='', additional_kwargs={'reasoning_content': 'Alright, the user is asking "what\'s 42 x 7?" So, I need to figure out how to respond using the tools provided. \n\nFirst, I check the available tools. There\'s a "multiply" function that takes two integers, a and b. That seems perfect for this calculation.\n\nI should call the multiply function with a=42 and b=7. I\'ll structure the JSON accordingly within the tool_call tags.\n\nDouble-checking the function\'s parameters to make sure I\'m using the correct keys and data types. Everything looks good, so I\'ll format the response as specified.\n', 'tool_calls': [{'id': '1w3ed42tg', 'function': {'arguments': '{"a":42,"b":7}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 154, 'prompt_tokens': 201, 'total_tokens': 355, 'completion_ti

In [42]:
print(f"**Agent**: {response['messages'][-1].content}")

**Agent**: The result of multiplying 42 by 7 is **294.


In [ ]:
response = agent.invoke({"messages": [{"role": "user", "content": "How is the weather in Berlin?"}]})
print(response)
print(f"**Agent**: {response['messages'][-1].content}")

2025-08-10 22:43:16,880 - httpx - INFO - _client.py - _send_single_request:1025 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-10 22:43:18,695 - httpx - INFO - _client.py - _send_single_request:1025 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-10 22:43:20,363 - httpx - INFO - _client.py - _send_single_request:1025 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


{'messages': [HumanMessage(content='How is the weather in Berlin?', additional_kwargs={}, response_metadata={}, id='dd14e7a6-8f81-4074-964c-69f43c2edabe'), AIMessage(content='', additional_kwargs={'reasoning_content': "Okay, the user is asking about the weather in Berlin. I need to figure out which tool to use. Looking at the tools provided, there's one called get_weather_forcast which takes a location as input. That seems perfect for this query. I should call this function with 'Berlin' as the location parameter. So, I'll structure the tool call with the name as get_weather_forcast and the arguments as a JSON object containing loc: 'Berlin'. That should give the user the weather forecast they're asking for.\n", 'tool_calls': [{'id': '7017zze1e', 'function': {'arguments': '{"loc":"Berlin"}', 'name': 'get_weather_forcast'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 136, 'prompt_tokens': 200, 'total_tokens': 336, 'completion_time': 0.537596587, 'promp